## Downloading the Dataset

Download and extract the 'amazon_review_full_csv.tar.gz' file from https://drive.google.com/drive/u/0/folders/0Bz8a_Dbh9Qhbfll6bVpmNUtUcFdjYmF2SEpmZUZUcVNiMUw1TWN6RDV3a0JHT3kxLVhVR2M

Make sure that 'train.csv' and 'test.csv' files are present in this directory

In [10]:
import os

assert os.path.exists('train.csv') and os.path.exists('test.csv')

In [11]:
import nltk
import pandas as pd
import csv
import sys
import spacy
import re
import random
import codecs 
# reload(sys)
# sys.setdefaultencoding('utf8')

In [12]:
random.seed(1357)
def read_input_file(input_file):
    lines = csv.reader(codecs.open(input_file, "r", encoding="utf-8"))
    lines = list(lines)
    random.shuffle(lines)
    labels, _, lines = list(zip(*lines))
    lines = list(zip(labels, lines))
    print (lines[:3])
    new_labels = []
    new_lines = []
#     for label, line in lines:
#         if int(label) < 3:
#             new_labels.append("0")
#             new_lines.append(line)
#         elif int(label) > 3:
#             new_labels.append("1")
#             new_lines.append(line)
    for label, line in lines:
        new_labels.append(label)
        new_lines.append(line)
        
    print (new_labels[:2], new_lines[:2])
    print(len(new_labels), len(new_lines))
    return new_labels, new_lines
                 

In [13]:
labels_train,  content_train = read_input_file("train.csv")
assert(len(labels_train) == len(content_train))
print (len(labels_train))

labels_dev, content_dev = labels_train[:7000], content_train[:7000]
keys_dev = ["dev"]* len(labels_dev)

labels_train, content_train = labels_train[7000:], content_train[7000:]
keys_train = ["train"]*len(labels_train)

[('3', 'got it to add to my kindle. have no real opinion about itgood to have with me when i want to read'), ('2', "First of all don't get me wrong I think the missions on this game are some of the best ever. But this game is NASTY there is no call for all of the half naked women and that such.... Why can't people make a game for clean half intellegent people who are above half naked women on GAMES and such. Catch that ON GAMES! I would rank this game a perfect 5 stars if it cut back on some language and the girls in it had on some what clothing. It makes me want to vomit. What kind of perv came up with the idea of doing this. There are people who would like a game like this without the cursing and sexual content. I dont appreciate the way people look at teens now. .... Gamers do not support games such as Vice City. ...."), ('2', 'I\'ve been a great fan of J.J. for some 25 years and possess some 15 of his albums. Also I am a great appreciator of Clapton as a guitarist but not necessari

In [14]:
labels_test, content_test = read_input_file("test.csv")
keys_test = ["test"]*len(labels_test)
assert(len(labels_test) == len(content_test))
print (len(labels_test))

[('3', "This book was enjoyable enough, but I felt like it was more like a movie script--written for mass appeal, predictable. There wasn't much depth to any of the characters. I wasn't bored, but I won't be reading it again, like I do most books I enjoy. I will probably see the movie, just because it seems like this book may be better as a movie--so I can see some emotion in the characters."), ('1', 'Not Pretzel Salt or even close. It is not white and course crystals; but clearish, thin and flat crytstals. Just threw out. Nothing like the great pretzel salt from Nuts.com.'), ('4', "If you like opeth I don't see how you can't like this. Two sets plus a making of. This DVD has great sound.")]
['3', '1'] ["This book was enjoyable enough, but I felt like it was more like a movie script--written for mass appeal, predictable. There wasn't much depth to any of the characters. I wasn't bored, but I won't be reading it again, like I do most books I enjoy. I will probably see the movie, just be

In [15]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

def tokenize(text) :
    #text = " ".join(text)
    text = text.replace("-LRB-", '')
    text = text.replace("-RRB-", " ")
    #text = re.sub(r'\W', ' ', text)
    #text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    tokens = " ".join([t.text.lower() for t in nlp((text))])
    return tokens

labels_train = [int(i)-1 for i in labels_train]
content_train = [tokenize(i) for i in content_train]


labels_test = [int(i)-1 for i in labels_test]
content_test = [tokenize(i) for i in content_test]


labels_dev = [int(i)-1 for i in labels_dev]
content_dev = [tokenize(i) for i in content_dev]

#assert(len(labels) == len(content))



In [16]:
print(labels_train[:3])
print(content_train[:3])
labels = labels_train + labels_dev + labels_test
content = content_train + content_dev + content_test
keys = keys_train + keys_dev + keys_test

[0, 1, 1]
['trailer park trash . garth brooks is far superior at the business of being garth , than he could even pretend to be as a musician .', "i was looking for engaging software for my 7th grade son to practice and increase understanding of math concepts . there 's plenty of practice here , but no explanation of how to figure out problems that were solved incorrectly . not helpful as a tutorial .", "i was very excited about getting wireless headphones . i had looked at a number of vendors and models and selected this one . i was immediately disappointed . i do n't know about all wireless headphones , but these have significant static noise at any distance . also , i can only get about 20 feet away before the noise is so bad i ca n't hear the music anymore . i would suggest looking at other wireless headphones if you 're looking for some ."]


In [17]:
df = pd.DataFrame({'text' : content, 'label' : labels, 'exp_split' : keys})
df.to_csv('amazon_dataset.csv', index=False)

In [18]:
print(labels_train[:3])
print(content_train[:3])

[0, 1, 1]
['trailer park trash . garth brooks is far superior at the business of being garth , than he could even pretend to be as a musician .', "i was looking for engaging software for my 7th grade son to practice and increase understanding of math concepts . there 's plenty of practice here , but no explanation of how to figure out problems that were solved incorrectly . not helpful as a tutorial .", "i was very excited about getting wireless headphones . i had looked at a number of vendors and models and selected this one . i was immediately disappointed . i do n't know about all wireless headphones , but these have significant static noise at any distance . also , i can only get about 20 feet away before the noise is so bad i ca n't hear the music anymore . i would suggest looking at other wireless headphones if you 're looking for some ."]


In [19]:
%run "../preprocess_data_BC.py" --data_file amazon_dataset.csv --output_file ./vec_amazon.p --word_vectors_type fasttext.simple.300d --min_df 35

Vocabulary size :  56250
Found 38758 words in model out of 56250


In [20]:
print(set(labels_train))
print(set(labels_test))
print(set(labels_dev))

{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4}
{0, 1, 2, 3, 4}
